In [ ]:
import numpy as np
from numba import njit, prange


@njit(parallel=True)
def kuwahara_numba(img, radius):
    """
    Applies the Kuwahara filter to a color image.

    Args:
        img: Numpy array of shape (H, W, 3) - expected to be floats or ints
        radius: The size of the window (integer). Higher = more abstract.

    Returns:
        The filtered image.
    """
    h, w, c = img.shape

    output = np.zeros_like(img)

    for y in prange(radius, h - radius):
        for x in range(radius, w - radius):

            tl = img[y - radius : y + 1, x - radius : x + 1]

            tr = img[y - radius : y + 1, x : x + radius + 1]

            bl = img[y : y + radius + 1, x - radius : x + 1]

            br = img[y : y + radius + 1, x : x + radius + 1]

            regions = (tl, tr, bl, br)

            min_variance = 1e99
            best_mean = np.zeros(3, dtype=np.float64)

            for region in regions:

                mean_val = np.array(
                    [
                        np.mean(region[:, :, 0]),
                        np.mean(region[:, :, 1]),
                        np.mean(region[:, :, 2]),
                    ]
                )

                var_val = (
                    np.var(region[:, :, 0])
                    + np.var(region[:, :, 1])
                    + np.var(region[:, :, 2])
                )

                if var_val < min_variance:
                    min_variance = var_val
                    best_mean = mean_val

            output[y, x, 0] = best_mean[0]
            output[y, x, 1] = best_mean[1]
            output[y, x, 2] = best_mean[2]

    return output

In [4]:
from PIL import Image
import os


image_path = "../samples/f1.png"
img = Image.open(image_path)


img_array = np.array(img)


if len(img_array.shape) == 2:

    img_array = np.stack([img_array, img_array, img_array], axis=-1)
elif img_array.shape[2] == 4:

    img_array = img_array[:, :, :3]


img_float = img_array.astype(np.float64)

print(f"Image shape: {img_float.shape}")
print(f"Image dtype: {img_float.dtype}")
print(f"Image range: [{img_float.min():.1f}, {img_float.max():.1f}]")


print("\nApplying Kuwahara filter...")
result = kuwahara_numba(img_float, radius=12)
print("Filter applied successfully!")


result_uint8 = np.clip(result, 0, 255).astype(np.uint8)


output_dir = "../outputs"
os.makedirs(output_dir, exist_ok=True)


output_path = os.path.join(output_dir, "f1_kuwahara.png")
result_image = Image.fromarray(result_uint8)
result_image.save(output_path)

print(f"\nResult saved to: {output_path}")

Image shape: (2160, 3840, 3)
Image dtype: float64
Image range: [0.0, 255.0]

Applying Kuwahara filter...
Filter applied successfully!

Result saved to: ../outputs/f1_kuwahara.png
